In [1]:
! pip install -r requirements.txt -q

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import mlflow
import mlflow.xgboost
import shap


In [3]:
import boto3
import io

s3 = boto3.client(
    's3',
    endpoint_url='http://minio-service.kubeflow.svc.cluster.local:9000',
    aws_access_key_id='minio',
    aws_secret_access_key='minio123',
)

In [4]:
obj_test = s3.get_object(Bucket='sample-data', Key='data/data/application_test.csv')
test_df = pd.read_csv(io.BytesIO(obj_test['Body'].read()))

In [5]:
obj_train = s3.get_object(Bucket='sample-data', Key='data/data/application_train.csv')
train_df = pd.read_csv(io.BytesIO(obj_train['Body'].read()))

In [6]:
# Get numerical and categorical columns
numerical_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# Remove 'SK_ID_CURR' and 'TARGET' from numerical features as they are identifiers/target
if 'SK_ID_CURR' in numerical_features:
    numerical_features.remove('SK_ID_CURR')
if 'TARGET' in numerical_features:
    numerical_features.remove('TARGET')

print("Categorical features:", len(categorical_features))
print(categorical_features)
print("\nNumerical features:", len(numerical_features))
print(numerical_features)

Categorical features: 16
['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

Numerical features: 104
['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'

In [7]:
from src.utils import PreprocessFeatureSelector

# Prepare the data (remove target column for features)
X_train = train_df.drop(columns=["TARGET"])
X_test = test_df.copy()
y_train = train_df["TARGET"].values

data_version = "v1"
save_train_data_path = "data/processed/train"
save_test_data_path = "data/processed/test"

# Create and run the PreprocessFeatureSelector wrapper.
pfs = PreprocessFeatureSelector(X_train, X_test, y_train,
                                categorical_features, numerical_features,
                                data_version=data_version,
                                save_train_data_path=save_train_data_path,
                                save_test_data_path=save_test_data_path)
selected_train, selected_test, selected_features, excluded_features = pfs.run()

print("Final Selected Features:", selected_features)
print("Total Excluded Features:", excluded_features)

2025-04-15 02:10:33.467 | INFO     | src.utils:run:119 - 🚀 Starting preprocessing and filtering...
2025-04-15 02:11:05.937 | INFO     | src.utils:run:123 - ✅ Preprocessing complete.
2025-04-15 02:11:05.941 | INFO     | src.utils:run:125 - 🧮 Features before filtering: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',

Final Selected Features: ['CODE_GENDER', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'ORGANIZATION_TYPE', 'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3']
Total Excluded Features: ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CNT_CHILDREN', 'AMT_ANNUITY', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMI

In [8]:
print("Target value counts:")
print(selected_train['TARGET'].value_counts())
print("\nPercentage distribution:")
print(selected_train['TARGET'].value_counts(normalize=True) * 100)

Target value counts:
TARGET
0    282686
1     24825
Name: count, dtype: int64

Percentage distribution:
TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64


In [9]:
from src.trainer import UnderWritingTrainer

processed_train = "data/processed/train/processed_train_v1.csv"
processed_test = "data/processed/test/processed_test_v1.csv"

trained_model = UnderWritingTrainer.train_model(
    model_name="xgb",
    processed_train=processed_train,
    processed_test=processed_test,
    version="v2",
    experiment_name="XGB",
)


2025-04-15 02:13:43.329 | INFO     | src.trainer:train_model:198 - Start training underwriting model xgb with version v2
2025-04-15 02:13:43.864 | INFO     | src.trainer:train_model:227 - No categorical features provided for schema mapping.
[I 2025-04-15 02:13:43,896] A new study created in memory with name: no-name-bf6fafde-0c3c-4c13-adf6-d03e3454eb9b


[0]	validation_0-auc:0.63692
[1]	validation_0-auc:0.65568
[2]	validation_0-auc:0.67358
[3]	validation_0-auc:0.67652
[4]	validation_0-auc:0.67562
[5]	validation_0-auc:0.67637
[6]	validation_0-auc:0.67615
[7]	validation_0-auc:0.67793
[8]	validation_0-auc:0.68482
[9]	validation_0-auc:0.68417
[10]	validation_0-auc:0.68387
[11]	validation_0-auc:0.68700


/opt/conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:13:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[12]	validation_0-auc:0.69038
[13]	validation_0-auc:0.68967
[14]	validation_0-auc:0.69033
[15]	validation_0-auc:0.68970
[16]	validation_0-auc:0.68854
[17]	validation_0-auc:0.68867
[18]	validation_0-auc:0.68763
[19]	validation_0-auc:0.68874
[20]	validation_0-auc:0.68873
[21]	validation_0-auc:0.68926
[22]	validation_0-auc:0.68991
[23]	validation_0-auc:0.69005
[24]	validation_0-auc:0.69152
[25]	validation_0-auc:0.69142
[26]	validation_0-auc:0.69191
[27]	validation_0-auc:0.69205
[28]	validation_0-auc:0.69176
[29]	validation_0-auc:0.69187
[30]	validation_0-auc:0.69295
[31]	validation_0-auc:0.69343
[32]	validation_0-auc:0.69333
[33]	validation_0-auc:0.69355
[34]	validation_0-auc:0.69394
[35]	validation_0-auc:0.69357
[36]	validation_0-auc:0.69395
[37]	validation_0-auc:0.69406
[38]	validation_0-auc:0.69501
[39]	validation_0-auc:0.69473
[40]	validation_0-auc:0.69481
[41]	validation_0-auc:0.69519
[42]	validation_0-auc:0.69492
[43]	validation_0-auc:0.69515
[44]	validation_0-auc:0.69489
[45]	valid

[I 2025-04-15 02:13:44,799] Trial 0 finished with value: 0.9195323805342829 and parameters: {'max_depth': 2, 'learning_rate': 0.023899944848030215, 'n_estimators': 108, 'subsample': 0.5321515598126965, 'colsample_bytree': 0.8682664878529063}. Best is trial 0 with value: 0.9195323805342829.


...

[I 2025-04-15 02:14:04,928] Trial 8 finished with value: 0.9194836024258979 and parameters: {'max_depth': 6, 'learning_rate': 0.048001591951286615, 'n_estimators': 351, 'subsample': 0.79648882222405, 'colsample_bytree': 0.9330917986178469}. Best is trial 0 with value: 0.9195323805342829.


[0]	validation_0-auc:0.60091
[1]	validation_0-auc:0.61879
[2]	validation_0-auc:0.69981
[3]	validation_0-auc:0.70456
[4]	validation_0-auc:0.70283
[5]	validation_0-auc:0.71052
[6]	validation_0-auc:0.70904
[7]	validation_0-auc:0.71274
[8]	validation_0-auc:0.71637
[9]	validation_0-auc:0.71706
[10]	validation_0-auc:0.71842
[11]	validation_0-auc:0.71965
[12]	validation_0-auc:0.71952


/opt/conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:14:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[13]	validation_0-auc:0.71934
[14]	validation_0-auc:0.71991
[15]	validation_0-auc:0.72015
[16]	validation_0-auc:0.72042
[17]	validation_0-auc:0.72034
[18]	validation_0-auc:0.72030
[19]	validation_0-auc:0.72025
[20]	validation_0-auc:0.72088
[21]	validation_0-auc:0.72114
[22]	validation_0-auc:0.72135
[23]	validation_0-auc:0.72127
[24]	validation_0-auc:0.72120
[25]	validation_0-auc:0.72133
[26]	validation_0-auc:0.72147
[27]	validation_0-auc:0.72158
[28]	validation_0-auc:0.72168
[29]	validation_0-auc:0.72146
[30]	validation_0-auc:0.72152
[31]	validation_0-auc:0.72177
[32]	validation_0-auc:0.72191
[33]	validation_0-auc:0.72208
[34]	validation_0-auc:0.72205
[35]	validation_0-auc:0.72210
[36]	validation_0-auc:0.72200
[37]	validation_0-auc:0.72209
[38]	validation_0-auc:0.72198
[39]	validation_0-auc:0.72207
[40]	validation_0-auc:0.72213
[41]	validation_0-auc:0.72209
[42]	validation_0-auc:0.72213
[43]	validation_0-auc:0.72210
[44]	validation_0-auc:0.72216
[45]	validation_0-auc:0.72214
[46]	valid

[I 2025-04-15 02:14:09,114] Trial 9 finished with value: 0.9194836024258979 and parameters: {'max_depth': 8, 'learning_rate': 0.08876788185244802, 'n_estimators': 366, 'subsample': 0.8574498869342663, 'colsample_bytree': 0.540802535225285}. Best is trial 0 with value: 0.9195323805342829.
/opt/conda/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [02:14:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predi